# Get Distance Matrix 

In this tutorial we will demonstrate how to use google solver to solve the vehicle routing problem (VRP) and the capacitated vehicle routing problem (CVRP). 

Step 1: Determine our Distance Matrix:
First we will use google's APIs to get coordinates of the locations of the delivery locations and depot. Then we will run the functions created by Google OR-Tools to convert the coordinates into a distance matrix.
Follow the link for further details: https://developers.google.com/optimization/routing/vrp#distance_matrix_api 

In [1]:
from __future__ import division
from __future__ import print_function
import requests
import json
import urllib
import urllib.request


def create_data():
  """Creates the data."""
  data = {}
  data['API_key'] = 'SET_YOUR_API_KEY'
  data['addresses'] = ['3898+Jenifer+St+NW+Washington+DC', # depot
                     '5301+41st+St+NW+Washington+DC', #1
                     '5335+42nd+Pl+NW+Washington+DC', #2
                     '3901+Garrison+St+NW+Washington+DC', #3
                     '4117+Ingomar+St+NW+Washington+DC', #4
                     '4198+Military+Rd+NW+Washington+DC', #5
                     '5201+Reno+Rd+NW+Washington+DC', #6
                     '5048+Reno+Rd+NW+Washington+DC', #7
                     '3701+Fessenden+St+NW+Washington+DC', #8
                     '3701+Jocelyn+St+NW+Washington+DC', #9 
                     '3701+Jenifer+St+NW+Washington+DC', #10
                     '5311+Connecticut+Ave+NW+Washington+DC', #11
                     '5100+Connecticut+Ave+NW+Washington+DC', #12
                     '3998+Harrison+St+NW+Washington+DC', #13
                     '3998+Military+Rd+NW+Washington+DC', #14
                     '5015+42nd+St+NW+Washington+DC'#15
                      ]
  return data

Next we will run the functions to create a distance matrix from our coordinates. Please note that there is already documentation provided in the code (noted in those lines preceded by an #) 


From the documentation the code computes the distance matrix as follows: 

1. Divides the 16 addresses into two groups of six addresses and one group of four addresses.
2. For each group, build and send a request for the origin addresses in the group and all destination addresses.
3. Use the response to build the corresponding rows of the matrix, and concatenate the rows (Python lists).
4. Print list

In [2]:
def create_distance_matrix(data):
  addresses = data["addresses"]
  API_key = data["API_key"]
  # Distance Matrix API only accepts 100 elements per request, so get rows in multiple requests.
  max_elements = 100
  num_addresses = len(addresses) # 16 in this example.
  # Maximum number of rows that can be computed per request (6 in this example).
  max_rows = max_elements // num_addresses
  # num_addresses = q * max_rows + r (q = 2 and r = 4 in this example).
  q, r = divmod(num_addresses, max_rows)
  dest_addresses = addresses
  distance_matrix = []
  # Send q requests, returning max_rows rows per request.
  for i in range(q):
    origin_addresses = addresses[i * max_rows: (i + 1) * max_rows]
    response = send_request(origin_addresses, dest_addresses, API_key)
    distance_matrix += build_distance_matrix(response)

  # Get the remaining remaining r rows, if necessary.
  if r > 0:
    origin_addresses = addresses[q * max_rows: q * max_rows + r]
    response = send_request(origin_addresses, dest_addresses, API_key)
    distance_matrix += build_distance_matrix(response)
  return distance_matrix


The following function builds and sends a request for a given set of origin and destination addresses.
- The sub-function build_address_string concatenates addresses separated by the pipe character, '|'.
- The remaining code in the function assembles the parts of the request described above, and sends the request. The line response = json.loads(jsonResult) converts the raw result to a Python object.

In [3]:
def send_request(origin_addresses, dest_addresses, API_key):
  """ Build and send request for the given origin and destination addresses."""
  def build_address_str(addresses):
    # Build a pipe-separated string of addresses
    address_str = ''
    for i in range(len(addresses) - 1):
      address_str += addresses[i] + '|'
    address_str += addresses[-1]
    return address_str

  request = 'https://maps.googleapis.com/maps/api/distancematrix/json?units=imperial'
  origin_address_str = build_address_str(origin_addresses)
  dest_address_str = build_address_str(dest_addresses)
  request = request + '&origins=' + origin_address_str + '&destinations=' + \
                       dest_address_str + '&key=' + API_key
  jsonResult = urllib.request.urlopen(request).read()
  response = json.loads(jsonResult)
  return response

The following function builds rows of the distance matrix, using the response returned by the send_request function.
- The line: row_list = [row['elements'][j]['distance']['value'] for j in range(len(row['elements']))] extracts the distances between locations for a row of the response.
- If you want to create a time matrix, containing travel times between locations, replace 'distance' with 'duration' in the function build_distance_matrix.

In [4]:
def build_distance_matrix(response):
  distance_matrix = []
  for row in response['rows']:
    row_list = [row['elements'][j]['distance']['value'] for j in range(len(row['elements']))]
    distance_matrix.append(row_list)
  return distance_matrix

Print the Distance Matrix:
- The following code in the main function runs the program

In [5]:
def main():
  """Entry point of the program"""
  # Create the data.
  data = create_data()
  addresses = data['addresses']
  API_key = data['API_key']
  distance_matrix = create_distance_matrix(data)
  print(distance_matrix)
if __name__ == '__main__':
  main()

[[0, 321, 647, 783, 492, 644, 556, 513, 829, 656, 602, 735, 773, 529, 371, 834], [321, 0, 422, 666, 266, 417, 500, 683, 999, 1125, 1244, 960, 1021, 303, 194, 608], [685, 467, 0, 989, 380, 146, 864, 1047, 1362, 1245, 1364, 1081, 1483, 631, 314, 638], [344, 666, 1133, 0, 613, 988, 453, 410, 599, 996, 897, 936, 748, 362, 675, 634], [492, 266, 380, 613, 0, 376, 670, 853, 1169, 1391, 1268, 1227, 1119, 251, 460, 342], [540, 322, 146, 844, 376, 0, 718, 901, 1217, 1099, 1219, 935, 1337, 625, 168, 634], [218, 540, 1008, 453, 710, 862, 0, 183, 499, 769, 670, 709, 521, 573, 550, 942], [401, 833, 1191, 410, 781, 1045, 183, 0, 316, 726, 627, 667, 478, 530, 733, 899], [717, 1149, 1506, 599, 1097, 1361, 499, 316, 0, 835, 651, 631, 228, 846, 1048, 950], [973, 1405, 1301, 982, 1353, 1156, 755, 712, 676, 0, 117, 608, 448, 1102, 987, 1471], [602, 923, 1312, 897, 1094, 1167, 670, 627, 592, 436, 0, 231, 363, 1017, 998, 1386], [493, 814, 1081, 936, 985, 935, 582, 637, 631, 204, 231, 0, 402, 1056, 767, 1569]

# Solve VRP

In this section solve the Vehicle Routing Problem.

First we load the required libraries and input our required data. The data required is the following:
1. Distance matrix
2. Number of vehicles
3. Depot (the index of the depot, the locaiton where all vehicles start and end their routes)

In [6]:
from __future__ import print_function
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [13]:
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = [
        [
            0, 321, 791, 743, 492, 644, 618, 513, 822, 656, 596, 735, 773, 529, 372, 834
        ], 
        [
            321, 0, 422, 662, 266, 418, 500, 683, 992, 1127, 1188, 963, 1021, 303, 194, 608
        ], 
        [
            647, 471, 0, 991, 380, 147, 865, 1048, 1357, 1247, 1308, 1083, 1485, 774, 314, 638
        ], 
        [
            344, 666, 1135, 0, 613, 988, 449, 410, 592, 996, 903, 936, 748, 362, 676, 490
        ], 
        [
            492, 266, 592, 609, 0, 377, 670, 853, 1162, 1393, 1454, 1229, 1119, 251, 461, 342
        ],
        [
            540, 324, 147, 844, 377, 0, 718, 901, 1210, 1100, 1161, 936, 1338, 627, 167, 635
        ], 
        [
            218, 500, 1009, 453, 670, 862, 0, 183, 492, 769, 676, 709, 521, 573, 551, 942
        ], 
        [
            401, 683, 1192, 410, 853, 1045, 183, 0, 309, 726, 634, 667, 478, 530, 734, 899
        ], 
        [
            710, 992, 1501, 592, 1162, 1354, 492, 309, 0, 842, 741, 637, 235, 839, 1043, 943
        ], 
        [
            973, 1255, 1303, 982, 1353, 1156, 755, 712, 683, 0, 123, 608, 448, 1102, 989, 1471
        ],
        [
            596, 917, 1372, 903, 1274, 1225, 676, 634, 604, 430, 0, 225, 369, 1023, 1058, 1393
        ], 
        [
            493, 814, 1083, 936, 1307, 936, 709, 667, 637, 204, 225, 0, 402, 1056, 768, 1426
        ], 
        [
            768, 1049, 1485, 776, 1147, 1338, 549, 507, 235, 607, 505, 402, 0, 896, 1170, 1266
        ], 
        [
            464, 303, 629, 362, 251, 627, 573, 530, 839, 1115, 1023, 1056, 868, 0, 498, 369
        ], 
        [
            372, 519, 314, 676, 690, 167, 551, 734, 1043, 932, 994, 768, 1170, 727, 0, 802
        ], 
        [
            834, 608, 638, 490, 342, 635, 942, 899, 943, 1485, 1393, 1426, 1237, 369, 802, 0
        ],
    ]
    data['num_vehicles'] = 3
    data['depot'] = 2
    return data

Add a solution printer
- The function below prints the solution when called

In [14]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    max_route_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
    print('Maximum of the route distances: {}m'.format(max_route_distance))

#### Main Function
The following function does the following: 
1. Create the routing model: Creates the index manager (manager) and the routing model (routing). The method manager.IndexToNode converts the solver's internal indices to the numbers for locations. Location numbers correspond to the indices for the distance matrix.
2. Define the distance callback (subfunction): Creates the distance callback which returns the distance between locations, and passes it to the solver.  Creates the callback and registers it with the solver as transit_callback_index.
3. Defines the cost of the travel distances in this case: 
4. Employes a solver. See different solvers here- https://developers.google.com/optimization/routing/routing_options#first_sol_options 

In [ ]:
def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Distance constraint.
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        3000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)

if __name__ == '__main__':
    main()